In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('ml-1m-ae/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m-ae/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m-ae/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
training_set = pd.read_csv('ml-100k-ae/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k-ae/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype = 'int')

In [4]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [5]:
nb_users

943

In [6]:
nb_movies

1682

In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        if (data[id_users] == data[id_users+1]):
            id_movies = data[:,1]
        if (data[id_users] == data[id_users+1]):
            id_ratings = data[:,2]
        ratings = np.zeros(nb_movies)
        ratings[id_movies -1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
def convert_2(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies -1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [9]:
training_set = convert_2(training_set)
test_set = convert_2(test_set)

In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Creating the architecture of the NN

In [11]:
# We need to create a class to specify and define the behaviours of our AE (to give instructions)
# Object of this class will be an AE
# Inheritence to use all the functions of the parent class
# SAE -> Stacked Auto Encoders
# We have several hidden layers -> several encodings of the input vector features
class SAE(nn.Module):
    def __init__(self, ): # Put nothing after the comma, cause we have inheritence
        
        super(SAE, self).__init__() # Get the functions/methods of the module
        
        ## nn is module, Linear and Sigmoid are classes of the nn module
        
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        # The next function get the output of the last function as input
        # nb_movies - 20neuron - 10neuron - 20 neuron - nb_movies -> these are the layers
        
        self.activation = nn.Sigmoid()
    
    def forward(self,x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

### Training the SAE

In [12]:
nb_epoch = 150
for epoch in range (1, nb_epoch + 1):
    train_loss = 0
    
    #we will use RMSE at the end and RMSE is a float. That's why we use float 
    s = 0. # number of users who rated at least one movie
    
    for id_user in range (nb_users):
        
        # Add a fake dimension for functions
        # index of fake dimension is 0
        # we create batch
        input = Variable(training_set[id_user]).unsqueeze(0)
        
        target = input.clone()
        if torch.sum(target.data>0) > 0:
            output = sae(input)
            # GD won't be computed in target, it'll computed just in input vector
            target.require_grad = False
            
            # zero values don't have impact on the update of the different weights
            # before the loss we make them 0 to save comp power
            output[target == 0] = 0 
            
            # calculate the loss func.
            loss = criterion(output, target)
            
            # number of all movies divided by number of rated movies
            mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)
            
            # increase or decrease the weights? ->backward method
            # direction of the optimization
            loss.backward()
            
            # loss data is the square error, that's why we take sqrt to get one degree loss
            train_loss  += np.sqrt(loss.data*mean_corrector)
            
            # if user rated, increase s
            s += 1.
            
            
            optimizer.step() # intensity/amount of the updates
    print('epoch :' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch :1 loss: tensor(1.7709)
epoch :2 loss: tensor(1.0967)
epoch :3 loss: tensor(1.0535)
epoch :4 loss: tensor(1.0382)
epoch :5 loss: tensor(1.0310)
epoch :6 loss: tensor(1.0265)
epoch :7 loss: tensor(1.0238)
epoch :8 loss: tensor(1.0219)
epoch :9 loss: tensor(1.0206)
epoch :10 loss: tensor(1.0197)
epoch :11 loss: tensor(1.0189)
epoch :12 loss: tensor(1.0182)
epoch :13 loss: tensor(1.0179)
epoch :14 loss: tensor(1.0174)
epoch :15 loss: tensor(1.0171)
epoch :16 loss: tensor(1.0168)
epoch :17 loss: tensor(1.0166)
epoch :18 loss: tensor(1.0163)
epoch :19 loss: tensor(1.0167)
epoch :20 loss: tensor(1.0160)
epoch :21 loss: tensor(1.0158)
epoch :22 loss: tensor(1.0160)
epoch :23 loss: tensor(1.0157)
epoch :24 loss: tensor(1.0156)
epoch :25 loss: tensor(1.0157)
epoch :26 loss: tensor(1.0156)
epoch :27 loss: tensor(1.0154)
epoch :28 loss: tensor(1.0150)
epoch :29 loss: tensor(1.0127)
epoch :30 loss: tensor(1.0114)
epoch :31 loss: tensor(1.0089)
epoch :32 loss: tensor(1.0070)
epoch :33 loss: t

In [15]:
test_loss = 0

#we will use RMSE at the end and RMSE is a float. That's why we use float 
s = 0. # number of users who rated at least one movie
for id_user in range (nb_users): 
    
    # Add a fake dimension for functions
    # index of fake dimension is 0
    # we create batch
    input = Variable(training_set[id_user]).unsqueeze(0)
      
    target = input.clone()
    if torch.sum(target.data>0) > 0:
        output = sae(input)
        # GD won't be computed in target, it'll computed just in input vector
        target.require_grad = False
        # zero values don't have impact on the update of the different weights
        # before the loss we make them 0 to save comp power
        output[target == 0] = 0 

        # calculate the loss func.
        loss = criterion(output, target)

        # number of all movies divided by number of rated movies
        mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)

        # loss data is the square error, that's why we take sqrt to get one degree loss
        test_loss  += np.sqrt(loss.data*mean_corrector)

        # if user rated, increase s
        s += 1.
print('loss: ' + str(test_loss/s))

loss: tensor(0.9184)
